In [ ]:
import numpy as np
import pandas as pd
import warnings
import math

import zipfile
import io
import requests
from pathlib import Path
from google.colab import files

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
import warnings
warnings.simplefilter('ignore')

Data sorce: https://www.iea.org/data-and-statistics/data-product/world-energy-balances

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip = '/content/drive/MyDrive/IEA balance 2022/WIND.zip'
z = zipfile.ZipFile(zip, 'r')
z.extractall()

In [ ]:
zip = '/content/drive/MyDrive/IEA balance 2022/WBAL.zip'
z = zipfile.ZipFile(zip, 'r')
z.extractall()

In [ ]:
zip = '/content/drive/MyDrive/IEA balance 2022/WBIG1.zip'
z = zipfile.ZipFile(zip, 'r')
z.extractall()

In [ ]:
zip = '/content/drive/MyDrive/IEA balance 2022/WORLDBIG2.TXT.zip'
z = zipfile.ZipFile(zip, 'r')
z.extractall()

In [ ]:
ind=pd.read_csv('/content/WORLDBAL.TXT', header=None,sep=r"\s+")

In [ ]:
indicators =pd.read_csv('/content/WORLDIND.TXT', header=None,sep=r"\s+")

In [ ]:
Country_code = pd.read_excel('/content/drive/MyDrive/IEA balance 2022/страны.xlsx')

In [ ]:
# Final Table
result_oil = []

In [ ]:
ind1 = ind.rename(columns={0: "Country", 1: "Product" , 2: "Year", 3: "Flow", 4: "Unit", 5: "Value"})

In [ ]:
ind1['Product'] = ind1['Product'].replace('TOTAL','Total')
ind1['Product'] = ind1['Product'].replace('CRNGFEED','"Crude, NGL and feedstocks"')
ind1['Product'] = ind1['Product'].replace('TOTPRODS','Oil products')

In [ ]:
ind1['Flow'] = ind1['Flow'].replace('NECHEM','Memo: Non-energy use in chemical/petrochemical')
ind1['Flow'] = ind1['Flow'].replace('MARBUNK','International marine bunkers')
ind1['Flow'] = ind1['Flow'].replace('AVBUNK','International aviation bunkers')
ind1['Flow'] = ind1['Flow'].replace('DOMESAIR','Domestic aviation')
ind1['Flow'] = ind1['Flow'].replace('DOMESNAV','Domestic navigation')
ind1['Flow'] = ind1['Flow'].replace('ROAD','Road')
ind1['Flow'] = ind1['Flow'].replace('RAIL','Rail')
ind1['Flow'] = ind1['Flow'].replace('TOTIND','Industry')
ind1['Flow'] = ind1['Flow'].replace('RESIDENT','Residential')
ind1['Flow'] = ind1['Flow'].replace('COMMPUB','Commercial and public services')
ind1['Flow'] = ind1['Flow'].replace('MAINELEC','Main activity producer electricity plants')
ind1['Flow'] = ind1['Flow'].replace('HEMAINC','Main activity producer CHP plants')
ind1['Flow'] = ind1['Flow'].replace('ELAUTOE','Autoproducer electricity plants')
ind1['Flow'] = ind1['Flow'].replace('HEAUTOC','Autoproducer CHP plants')
ind1['Flow'] = ind1['Flow'].replace('HEMAINH','Main activity producer heat plants')
ind1['Flow'] = ind1['Flow'].replace('HEAUTOH','Autoproducer heat plants')
ind1['Flow'] = ind1['Flow'].replace('THEAT','Heat pumps')
ind1['Flow'] = ind1['Flow'].replace('TBOILER','Electric boilers')
ind1['Flow'] = ind1['Flow'].replace('TELE','Chemical heat for electricity production')
ind1['Flow'] = ind1['Flow'].replace('DISTLOSS','Losses')
ind1['Flow'] = ind1['Flow'].replace('TFC','Total final consumption')
ind1['Flow'] = ind1['Flow'].replace('INDPROD','Production')
ind1['Flow'] = ind1['Flow'].replace('TREFINER','Oil refineries')
ind1['Flow'] = ind1['Flow'].replace('TRANSFER','Transfers')
ind1['Flow'] = ind1['Flow'].replace('EXPORTS','Exports')
ind1['Flow'] = ind1['Flow'].replace('IMPORTS','Imports')

In [ ]:
IEA_data2 = ind1

In [ ]:
IEA_data2 = IEA_data2.loc[(IEA_data2['Unit'] == 'KTOE')]

In [ ]:
IEA_data2['Value'] = IEA_data2['Value'].replace('..',0).replace('x',0).replace('c',0)

In [ ]:
IEA_data2['Value'] = IEA_data2['Value'].apply(float)

# **TOTAL FINAL CONSUMPTION**

In [ ]:
TFC = IEA_data2.loc[(IEA_data2['Flow'] == 'Total final consumption')]
TFC_total = TFC.loc[(TFC['Product'] == 'Total')]
TFC_crude = TFC.loc[(TFC['Product'] == '"Crude, NGL and feedstocks"')]
TFC_products = TFC.loc[(TFC['Product'] == 'Oil products')]

In [ ]:
TFC_total['Parameter1'] = 'TFC'
TFC_total['Parameter2'] = 'Total'
result_oil.append(TFC_total)

TFC_crude['Parameter1'] = 'TFC'
TFC_crude['Parameter2'] = 'Crude oil'
result_oil.append(TFC_crude)

TFC_products['Parameter1'] = 'TFC'
TFC_products['Parameter2'] = 'Oil products'
result_oil.append(TFC_products)

# Production

In [ ]:
product = IEA_data2.loc[(IEA_data2['Flow'] == 'Production')]
product_oil = product.loc[(product['Product'] == '"Crude, NGL and feedstocks"')]

In [ ]:
product_oil['Parameter1'] = 'oil production'
product_oil['Parameter2'] = ''
result_oil.append(product_oil)

# Oil refinery

In [ ]:
refine = IEA_data2.loc[(IEA_data2['Flow'] == 'Oil refineries')]
refine_oil = refine.loc[(refine['Product'] == 'Oil products')]

In [ ]:
refine_oil['Parameter1'] = 'oil refinery'
refine_oil['Parameter2'] = ''
result_oil.append(refine_oil)

# EXPORT


In [ ]:
Export = IEA_data2.loc[(IEA_data2['Flow'] == 'Exports')]
Export_crude = Export.loc[(Export['Product'] == '"Crude, NGL and feedstocks"')]
Export_oil = Export.loc[(Export['Product'] == 'Oil products')]

In [ ]:
Export_crude['Parameter1'] = 'Export_crude_oil'
Export_crude['Parameter2'] = ''
result_oil.append(Export_crude)

Export_oil['Parameter1'] = 'Export_oil_products'
Export_oil['Parameter2'] = ''
result_oil.append(Export_oil)

# IMPORT

In [ ]:
Import = IEA_data2.loc[(IEA_data2['Flow'] == 'Imports')]
Import_crude = Import.loc[(Import['Product'] == '"Crude, NGL and feedstocks"')]
Import_oil = Import.loc[(Import['Product'] == 'Oil products')]

In [ ]:
Import_crude['Parameter1'] = 'Import_crude_oil'
Import_crude['Parameter2'] = ''
result_oil.append(Import_crude)

Import_oil['Parameter1'] = 'Import_oil_products'
Import_oil['Parameter2'] = ''
result_oil.append(Import_oil)

# RESULTS OIL


In [ ]:
result_concat_oil = pd.concat(result_oil)

In [ ]:
result_concat_oil_1 = result_concat_oil.drop(columns=['Unit', 'Product','Flow'])

In [ ]:
result_concat_oil_1['Value'] = result_concat_oil_1['Value'].replace('..',0).apply(float)

In [ ]:
result_table_oil = pd.pivot_table(result_concat_oil_1, 'Value', ['Country', 'Parameter1', 'Parameter2'], 'Year', aggfunc='sum').reset_index()


In [ ]:
result_table_oil = pd.concat([result_table_oil.loc[:,['Country',	'Parameter1',	'Parameter2']] , result_table_oil.loc[:,1970:]], axis=1)



In [ ]:
result_table_sorted_oil = result_table_oil.sort_values(['Parameter1', 'Parameter2', 'Country'])

In [ ]:
# Make one column Parameter in standard form
result_table_sorted_oil['Parameter1'] = result_table_sorted_oil['Parameter1'].str.replace(',','')
result_table_sorted_oil['Parameter1'] = result_table_sorted_oil['Parameter1'] + '_' + result_table_sorted_oil['Parameter2']
result_table_sorted_oil['Parameter1'] = 'OIL_' + result_table_sorted_oil['Parameter1']
result_table_sorted_oil = result_table_sorted_oil.drop(columns = 'Parameter2')
result_table_sorted_oil = result_table_sorted_oil.rename(columns = {'Parameter1': 'Parameter'})

In [ ]:
def func(x):
    if x.endswith('_'):
        x = x[:-1]
    return x

result_table_sorted_oil['Parameter'] = result_table_sorted_oil['Parameter'].apply(func)

In [ ]:
result_table_sorted_oil.iloc[:,:-1] = result_table_sorted_oil.iloc[:,:-1].replace(float('nan'), 0).replace('..', 0)

In [ ]:
estimations_1 = result_table_sorted_oil

In [ ]:
for i in range(estimations_1.shape[0]):
  if estimations_1.iloc[i, -4] == estimations_1.iloc[i, -3] == estimations_1.iloc[i, -2] == 0:
    estimations_1.iloc[i, -1] = 0
  else:
    if estimations_1.iloc[i, -1] == 0:
      estimations_1.iloc[i, -1] = float('nan')

In [ ]:
estimations_1.to_excel('IEA_OIL.xlsx', index = False)